In [2]:
import pandas as pd
import os
os.chdir(r"C:\Users\BingruYan\Data Analysis\练习\Titanic")

In [6]:
# 载入数据
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [8]:
# 处理缺失值
train['Age'].fillna(train['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)

train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)

train['Cabin'].fillna('Unknown', inplace=True)
test['Cabin'].fillna('Unknown', inplace=True)b

C:\Users\BingruYan\AppData\Local\Temp\ipykernel_3276\1019671117.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Age'].fillna(train['Age'].median(), inplace=True)
C:\Users\BingruYan\AppData\Local\Temp\ipykernel_3276\1019671117.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

In [12]:
# 提取头衔
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# 归并稀有头衔
for dataset in [train, test]:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace(['Mlle', 'Ms'], 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

# 创建家庭规模与单身特征
for dataset in [train, test]:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1


<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
C:\Users\BingruYan\AppData\Local\Temp\ipykernel_3276\1554336877.py:2: SyntaxWarning: invalid escape sequence '\.'
  train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
C:\Users\BingruYan\AppData\Local\Temp\ipykernel_3276\1554336877.py:3: SyntaxWarning: invalid escape sequence '\.'
  test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [14]:
# 性别与港口等特征编码
train = pd.get_dummies(train, columns=['Sex', 'Embarked', 'Title'], drop_first=True)
test = pd.get_dummies(test, columns=['Sex', 'Embarked', 'Title'], drop_first=True)


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score

# 训练集和标签
X = train.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y = train['Survived']

# 简单分割测试
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# 模型训练
models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=0),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=0),
    "Support Vector Machine": SVC(probability=True)
}

# 比较各模型表现
for name, model in models.items():
    model.fit(X_train, y_train)
    score = cross_val_score(model, X, y, cv=5).mean()
    print(f"{name}: {score:.4f}")


C:\Users\BingruYan\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\BingruYan\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

Logistic Regression: 0.8193
Random Forest: 0.8070
Gradient Boosting: 0.8294
Support Vector Machine: 0.6713


In [18]:
from sklearn.model_selection import GridSearchCV

# 网格搜索参数
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(GradientBoostingClassifier(random_state=0), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)


Best parameters: {'max_depth': 4, 'min_samples_split': 2, 'n_estimators': 100}
Best cross-validation score: 0.8417613458037788


In [20]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=200)),
        ('rf', RandomForestClassifier(n_estimators=300, max_depth=8, random_state=0)),
        ('gb', GradientBoostingClassifier(n_estimators=100, max_depth=4,min_samples_split=2, random_state=0))
    ],
    voting='soft'
)

voting_clf.fit(X_train, y_train)
voting_score = cross_val_score(voting_clf, X, y, cv=5).mean()
print("Voting Classifier score:", voting_score)


C:\Users\BingruYan\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\BingruYan\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

Voting Classifier score: 0.8338961772644529


In [32]:
test["Fare"].fillna(X_test["Fare"].median(),inplace=True)

C:\Users\BingruYan\AppData\Local\Temp\ipykernel_3276\1721981417.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test["Fare"].fillna(X_test["Fare"].median(),inplace=True)


In [34]:
# 测试集预测
X_test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
predictions = voting_clf.predict(X_test)

# 生成提交文件
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': predictions})
submission.to_csv('submission_ChatGPT.csv', index=False)
